In [10]:
import os
import sys
from pathlib import Path
import numpy as np
import pandas as pd
import librosa
from scipy.stats import entropy
from tqdm import tqdm
import soundfile as sf
from multiprocessing import Pool, cpu_count
from collections import defaultdict
import pyarrow as pa
import pyarrow.parquet as pq

In [4]:
# === Configuration ===
HOME = Path(os.environ["HOME"])
REPO_ROOT = HOME / "Uni-stuff/semester-2/applied_Ml/reef_zmsc"
MANIFEST = REPO_ROOT / "data/manifests/sample_50k_stratified.parquet"
OUTPUT_BASE = REPO_ROOT / "data/features/embeds_ecoacoustic_50k"

In [3]:
SAMPLE_RATE = 16000
N_FFT = 2048
HOP_LENGTH = 512
CLIP_LIMIT = None  # Set to None to process all 50K, or a number to limit
N_WORKERS = max(1, cpu_count() - 2)
BATCH_SIZE = 1000  # Write to disk every 1000 clips

In [5]:
def load_audio_clip(wav_path, start_s, end_s, sr=SAMPLE_RATE):
    """Load a specific time slice from a WAV file."""
    try:
        with sf.SoundFile(wav_path) as f:
            original_sr = f.samplerate
            start_frame = int(start_s * original_sr)
            duration_s = end_s - start_s
            frames_to_read = int(duration_s * original_sr)
            
            f.seek(start_frame)
            audio = f.read(frames_to_read)
            
            if len(audio.shape) > 1:
                audio = audio.mean(axis=1)
            
            if original_sr != sr:
                audio = librosa.resample(audio, orig_sr=original_sr, target_sr=sr)
            
            return audio.astype(np.float32)
    
    except Exception as e:
        duration_s = end_s - start_s
        return np.zeros(int(sr * duration_s), dtype=np.float32)

In [11]:
def extract_ecoacoustic_features(audio, sr):
    """
    Extract streamlined ecoacoustic indices (~15 features).
    Features are normalized to comparable scales for fusion with YAMNet.
    """
    # Compute spectrogram once
    S = np.abs(librosa.stft(audio, n_fft=N_FFT, hop_length=HOP_LENGTH))
    
    features = {}
    
    # === Spectral Features (6) - Normalize to [0, 1] ===
    centroid = librosa.feature.spectral_centroid(S=S, sr=sr, hop_length=HOP_LENGTH)
    bandwidth = librosa.feature.spectral_bandwidth(S=S, sr=sr, hop_length=HOP_LENGTH)
    rolloff = librosa.feature.spectral_rolloff(S=S, sr=sr, hop_length=HOP_LENGTH)
    flatness = librosa.feature.spectral_flatness(S=S, hop_length=HOP_LENGTH)
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr, hop_length=HOP_LENGTH)
    
    # Normalize by Nyquist frequency (sr/2 = 8000 Hz for 16kHz audio)
    nyquist = sr / 2
    features['spectral_centroid_mean'] = float(np.mean(centroid) / nyquist)
    features['spectral_centroid_std'] = float(np.std(centroid) / nyquist)
    features['spectral_bandwidth_mean'] = float(np.mean(bandwidth) / nyquist)
    features['spectral_rolloff_mean'] = float(np.mean(rolloff) / nyquist)
    features['spectral_flatness_mean'] = float(np.mean(flatness))
    features['spectral_contrast_mean'] = float(np.mean(contrast) / 40.0)
    
    # === Acoustic Complexity Index (1) ===
    diffs = np.abs(np.diff(S, axis=1))
    aci_raw = np.sum(diffs) / (np.sum(S[:, :-1]) + 1e-10)
    features['aci'] = float(np.clip(aci_raw / 2.0, 0, 1))
    
    # === Entropy Features (2) ===
    mean_spectrum = np.mean(S, axis=1)
    mean_spectrum = mean_spectrum / (np.sum(mean_spectrum) + 1e-10)
    spectral_H = entropy(mean_spectrum + 1e-10)
    features['spectral_entropy'] = float(spectral_H / 8.0)
    
    envelope = np.abs(audio)
    envelope = envelope / (np.sum(envelope) + 1e-10)
    temporal_H = entropy(envelope + 1e-10)
    features['temporal_entropy'] = float(temporal_H / 15.0)
    
    # === Zero Crossing Rate (1) ===
    zcr = librosa.feature.zero_crossing_rate(audio, hop_length=HOP_LENGTH)
    features['zcr_mean'] = float(np.mean(zcr))
    
    # === Energy Features (3) ===
    rms = librosa.feature.rms(y=audio, hop_length=HOP_LENGTH).flatten()
    rms_mean = np.mean(rms)
    rms_max = np.max(rms)
    
    features['rms_mean'] = float(rms_mean)
    features['rms_std'] = float(np.std(rms))
    
    dynamic_range_raw = 20 * np.log10(rms_max / (rms_mean + 1e-10))
    features['dynamic_range_db'] = float(np.clip(dynamic_range_raw / 40.0, 0, 1))
    
    # === SNR Estimate (1) ===
    noise_level = np.percentile(rms, 10)
    signal_level = np.mean(rms)
    snr_raw = 20 * np.log10(signal_level / (noise_level + 1e-10))
    features['snr_db'] = float(np.clip(snr_raw / 40.0, 0, 1))
    
    # === Low/Mid/High Frequency Energy (3) ===
    freq_bins = librosa.fft_frequencies(sr=sr, n_fft=N_FFT)
    low_mask = freq_bins < 1000
    mid_mask = (freq_bins >= 1000) & (freq_bins < 3000)
    high_mask = freq_bins >= 3000
    
    total_energy = np.sum(S) + 1e-10
    features['low_freq_energy'] = float(np.sum(S[low_mask, :]) / total_energy)
    features['mid_freq_energy'] = float(np.sum(S[mid_mask, :]) / total_energy)
    features['high_freq_energy'] = float(np.sum(S[high_mask, :]) / total_energy)
    
    return features

In [12]:
def detect_logger_date(wav_path: str):
    """Extract logger ID and date from path - handles PAPCA and PAPCA_test."""
    try:
        parts = Path(wav_path).parts
        
        # Find any part that starts with "PAPCA"
        papca_idx = -1
        for idx, part in enumerate(parts):
            if part.startswith("PAPCA"):
                papca_idx = idx
                break
        
        if papca_idx >= 0 and papca_idx + 2 < len(parts):
            logger = parts[papca_idx + 1]
            date = parts[papca_idx + 2]
            
            if logger and date and logger.strip() and date.strip() and logger != "wav":
                return logger, date
        
    except Exception as e:
        pass
    
    return "unknown", "unknown"


def shard_path(out_root: Path, logger: str, date: str) -> Path:
    """Determines which Parquet shard to write this clip's features to."""
    logger = logger.strip() if logger else "unknown"
    date = date.strip() if date else "unknown"
    
    # Match YAMNet structure: OUT_ROOT/PAPCA/<logger>/<date>/features.parquet
    return out_root / "PAPCA" / logger / date / "features.parquet"


def process_clip(args):
    """Process a single clip from manifest."""
    idx, wav_path, start_s, end_s = args
    
    try:
        # Detect logger and date from path
        logger, date = detect_logger_date(wav_path)
        
        # Load audio
        audio = load_audio_clip(wav_path, start_s, end_s, SAMPLE_RATE)
        
        # Extract features
        features = extract_ecoacoustic_features(audio, SAMPLE_RATE)
        
        # Add metadata
        features['filepath'] = wav_path
        features['start_s'] = start_s
        features['end_s'] = end_s
        features['logger'] = logger
        features['date'] = date
        
        return features
    
    except Exception as e:
        print(f"✗ Error processing clip {idx}: {e}")
        return None


def save_batch(batch_data, output_base):
    """Save a batch of features grouped by logger/date using efficient append."""
    if not batch_data:
        return 0
    
    # Group by logger and date
    grouped = defaultdict(list)
    for features in batch_data:
        key = (features['logger'], features['date'])
        grouped[key].append(features)
    
    saved_count = 0
    for (logger, date), features_list in grouped.items():
        # Use shard_path function to get the correct path structure
        output_path = shard_path(output_base, logger, date)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Convert to PyArrow Table
        new_table = pa.Table.from_pylist(features_list)
        
        # Append or create new file
        try:
            if output_path.exists():
                existing_table = pq.read_table(output_path)
                combined_table = pa.concat_tables([existing_table, new_table])
                pq.write_table(combined_table, output_path, compression="snappy")
            else:
                pq.write_table(new_table, output_path, compression="snappy")
            
            saved_count += len(features_list)
        except Exception as e:
            print(f"⚠️ Error saving to {output_path}: {e}")
    
    return saved_count

In [13]:
def main():
    print("=" * 70)
    print("ECOACOUSTIC FEATURES EXTRACTION - 50K STRATIFIED SAMPLE")
    print("=" * 70)
    
    # Load manifest
    print(f"\n📂 Loading manifest from: {MANIFEST}")
    df = pd.read_parquet(MANIFEST)
    print(f"   Total clips in manifest: {len(df):,}")
    
    # Determine filepath column
    PATH_COL = "filepath" if "filepath" in df.columns else "wav_path"
    assert PATH_COL in df.columns, f"Missing required column: {PATH_COL}"
    
    # Check for required columns
    required_cols = [PATH_COL, 'start_s']
    for col in required_cols:
        assert col in df.columns, f"Missing required column: {col}"
    
    # Handle end_s column (might not exist in stratified sample)
    if 'end_s' not in df.columns:
        print("   ℹ️  'end_s' column not found, computing from 'duration_s'")
        if 'duration_s' in df.columns:
            df['end_s'] = df['start_s'] + df['duration_s']
        else:
            print("   ⚠️  Assuming 10s clip duration")
            df['end_s'] = df['start_s'] + 10.0
    
    # Limit clips if specified
    if CLIP_LIMIT and len(df) > CLIP_LIMIT:
        print(f"   ⚠️  Limiting to {CLIP_LIMIT:,} clips for testing")
        df = df.sample(n=CLIP_LIMIT, random_state=42).reset_index(drop=True)
    
    print(f"\n📊 Processing {len(df):,} clips")
    print(f"   Batch size: {BATCH_SIZE} clips")
    print(f"   Workers: {N_WORKERS}")
    print(f"   Output: {OUTPUT_BASE.relative_to(REPO_ROOT)}")
    
    # Prepare data
    clip_data = [
        (idx, row[PATH_COL], row['start_s'], row['end_s'])
        for idx, row in df.iterrows()
    ]
    
    # Process in batches
    print(f"\n🔧 Extracting ecoacoustic features...")
    
    total_saved = 0
    batch_buffer = []
    
    with Pool(N_WORKERS) as pool:
        with tqdm(total=len(clip_data), desc="Processing clips", unit="clip") as pbar:
            for result in pool.imap(process_clip, clip_data, chunksize=20):
                if result is not None:
                    batch_buffer.append(result)
                
                # Save batch when buffer is full
                if len(batch_buffer) >= BATCH_SIZE:
                    saved = save_batch(batch_buffer, OUTPUT_BASE)
                    total_saved += saved
                    batch_buffer = []
                    pbar.set_postfix({'saved': total_saved})
                
                pbar.update(1)
            
            # Save remaining clips
            if batch_buffer:
                saved = save_batch(batch_buffer, OUTPUT_BASE)
                total_saved += saved
    
    print(f"\n✅ Successfully processed and saved {total_saved:,} clips")
    
    # Collect and display statistics
    print(f"\n📊 Collecting feature statistics...")
    
    all_features = []
    for parquet_file in OUTPUT_BASE.rglob("features.parquet"):
        df_features = pd.read_parquet(parquet_file)
        all_features.append(df_features)
    
    if all_features:
        combined = pd.concat(all_features, ignore_index=True)
        
        # Feature columns (exclude metadata)
        feature_cols = [c for c in combined.columns 
                       if c not in ['filepath', 'logger', 'date', 'start_s', 'end_s']]
        
        print(f"\n📐 Feature dimensions: {len(feature_cols)} features")
        print("\nFeature names:")
        for i, col in enumerate(feature_cols, 1):
            print(f"   {i:2d}. {col}")
        
        # Save statistics
        stats_path = OUTPUT_BASE / "feature_statistics.csv"
        stats = combined[feature_cols].describe()
        stats.to_csv(stats_path)
        print(f"\n💾 Statistics saved: {stats_path.relative_to(REPO_ROOT)}")
        
        # Show distribution by logger/date
        print(f"\n📦 Files created by logger/date:")
        grouped = combined.groupby(['logger', 'date']).size().reset_index(name='count')
        for _, row in grouped.iterrows():
            logger, date, count = row['logger'], row['date'], row['count']
            output_path = shard_path(OUTPUT_BASE, logger, date)
            rel_path = output_path.relative_to(REPO_ROOT)
            print(f"   ✓ {logger}/{date}: {count:>5} clips → {rel_path}")
        
        print(f"\n📈 Summary:")
        print(f"   Total clips processed: {len(combined):,}")
        print(f"   Unique loggers: {combined['logger'].nunique()}")
        print(f"   Unique dates: {combined['date'].nunique()}")
        print(f"   Features per clip: {len(feature_cols)}")
    
    print("\n" + "=" * 70)
    print("✅ ECOACOUSTIC EXTRACTION COMPLETE")
    print("=" * 70)


if __name__ == "__main__":
    main()

ECOACOUSTIC FEATURES EXTRACTION - 50K STRATIFIED SAMPLE

📂 Loading manifest from: /home/sparch/Uni-stuff/semester-2/applied_Ml/reef_zmsc/data/manifests/sample_50k_stratified.parquet
   Total clips in manifest: 50,000
   ℹ️  'end_s' column not found, computing from 'duration_s'

📊 Processing 50,000 clips
   Batch size: 1000 clips
   Workers: 18
   Output: data/features/embeds_ecoacoustic_50k

🔧 Extracting ecoacoustic features...


Processing clips: 100%|██████████| 50000/50000 [07:36<00:00, 109.49clip/s, saved=5e+4] 



✅ Successfully processed and saved 50,000 clips

📊 Collecting feature statistics...

📐 Feature dimensions: 17 features

Feature names:
    1. spectral_centroid_mean
    2. spectral_centroid_std
    3. spectral_bandwidth_mean
    4. spectral_rolloff_mean
    5. spectral_flatness_mean
    6. spectral_contrast_mean
    7. aci
    8. spectral_entropy
    9. temporal_entropy
   10. zcr_mean
   11. rms_mean
   12. rms_std
   13. dynamic_range_db
   14. snr_db
   15. low_freq_energy
   16. mid_freq_energy
   17. high_freq_energy

💾 Statistics saved: data/features/embeds_ecoacoustic_50k/feature_statistics.csv

📦 Files created by logger/date:
   ✓ 2802/20080226:    89 clips → data/features/embeds_ecoacoustic_50k/PAPCA/2802/20080226/features.parquet
   ✓ 2802/20080227:    77 clips → data/features/embeds_ecoacoustic_50k/PAPCA/2802/20080227/features.parquet
   ✓ 2802/20080228:   104 clips → data/features/embeds_ecoacoustic_50k/PAPCA/2802/20080228/features.parquet
   ✓ 2802/20080229:   167 clips →